# **Configurando as bibliotecas do projeto**

## *Instalação de bibliotecas*

In [3]:
!pip install Ta-Lib

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


## *Importação de bibliotecas*

In [5]:
'''
    Essa célula será usada para a importação de bibliotecas.
'''

# Instala a biblioteca que será utilizada para se obter os dados financeiros.
import yfinance as yf
# Importa a biblioteca que será utilizada para a manipulação de DataFrames.
import pandas as pd
# Importa a biblioteca que será utilizada para lidar com objetos do tipo "datetime".
from datetime import datetime, timedelta
# Importa a biblioteca que será responsável pela tipagem de funções.
from typing import Union, Optional
# Importa a biblioteca que será utilizada para a manipulação de arrays e também para o uso de algumas funções matemáticas.
import numpy as np
# Importa a biblioteca que será utilizada para calcular métricas de análise técnica.
import talib
# Importa o módulo que será utilizado para escalar as features do DataFrame final.
from sklearn.preprocessing import MinMaxScaler
# Importa o módulo que será utilizado para criar e treinar o modelo LSTM. 
from tensorflow.keras.models import Sequential
# Importa as camadas que serão utilizadas para construir a arquitetura do modelo LSTM, incluindo LSTM, camadas densas e dropout.
from tensorflow.keras.layers import LSTM, Dense, Dropout
# Importa a biblioteca que será utilizada para criar gráficos interativos.
import plotly.graph_objects as go

2024-09-10 22:08:12.282510: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-10 22:08:12.498480: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-10 22:08:12.733461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 22:08:12.918254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-10 22:08:12.980177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-10 22:08:13.335269: I tensorflow/core/platform/cpu_feature_gu

# **Data Engineering**

### *Configurações iniciais*

In [6]:
'''
    Essa célula será usada para se obter os tickets das companhias que fazem parte do S&P 500, com base em dados da wikipedia.
'''

# Salva em uma variável a url que contém a tabela com os tickets das companhias.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Lê todas as tabelas presentes na url acima.
sp500_table = pd.read_html(url)

# Salva a coluna "Symbol" da primeira tabela em uma variável (tal coluna contém todos os tickets das companhias que fazem parte do S&P 500).
sp500_tickets = sp500_table[0]['Symbol']

# Transforma os tickets obtidos em uma lista.
sp500_tickets = sp500_tickets.tolist()

# Exibe a lista de tickets criada acima.
sp500_tickets

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG'

In [7]:
'''
    Essa célula será usada para definir o intervalo de tempo dos dados que serão coletados, especificando a data de início e a data de fim.
'''

# Define a data inicial cujos dados serão coletados.
start_date = datetime(2019,1,2).date()
# Define a data final cujos dados serão coletados.
end_date = datetime(2023,12,30).date()

In [8]:
'''
    Essa célula será usada para criar um dicionário chamado setup, que contém os parâmetros principais para a coleta de dados, 
    incluindo a lista de tickers das companhias do S&P 500 e o intervalo de tempo.
'''

# Cria um dicionário para guardar alguns parâmetros importantes para a coleta de dados.
setup = {
    "tickers": sp500_tickets,
    "start_date": start_date,
    "end_date": end_date,
    # Número de tickers com maior "V Value" que serão utilizados no treinamento do modelo LSTM.
    "tickers_to_select": 1,
    "test_initial_day": np.datetime64('2023-12-01')
}

### *Obtendo dados dos tickers via yfinance (Open, High, Low, Close, Adj Close, Volume)*

In [9]:
'''
    Essa célula será usada para a obtenção de alguns dados (Open, High, Low, Close, Adj Close, Volume) 
    dos tickers presentes na variável de configuração setup.
'''

# Cria um array que guardará os DataFrames que contém os dados referentes a cada um dos tickers presentes na variável de configuração setup.
data_array = np.empty(len(setup['tickers']), dtype=object)

# Itera sobre cada um dos tickers presentes na variável de configuração setup.
for i, ticker in enumerate(setup['tickers']):
    # Faz o download dos dados do ticker em questão
    ticker_data = yf.download(ticker, setup['start_date'], setup['end_date'])
    # Adiciona um nome ao DataFrame do ticker em questão.
    ticker_data.name=ticker
    # Salva o DataFrame do ticker em questão na variável "data_array".
    data_array[i] = ticker_data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2019-01-02 -> 2023-12-30)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

### *Filtrando os dados obtidos*

In [10]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames não vazios.
'''

# Cria um filtro para identificar quais DataFrames presentes em "data_array" são vazios, isto é, não possuem nenhum índice.
void_filter = np.array([df.index.empty for df in data_array])

# Aplica a negação do filtro gerado acima (ou seja, filtra os DataFrames que não são vazios) no array "data_array" e armazena o resultado 
# na variável "filtered_data_array". 
filtered_data_array = data_array[~void_filter]

# Exibe uma mensagem que informa quantos tickers foram removidos na filtragem realizada acima.
print(f"Foram removidos {len(data_array) - len(filtered_data_array)} tickers que estavam vazios.")

Foram removidos 6 tickers que estavam vazios.


In [11]:
'''
    Essa célula será responsável por filtrar a variável "data_array" de modo que esta só contenha DataFrames cujos índices se iniciem em 
    setup['start_date'].
'''

# Armazena o tamanho atual do array "filtered_data_array" para que se tenha ideia ao final dessa célula, de quantos tickers 
# foram removidos com o filtro que será aplicado aqui.
filtered_data_array_initial_len = len(filtered_data_array)

# Cria um filtro para saber quais dos DataFrames presentes em "filtered_data_array" possuem datas iniciais (isto é, índices de posição 0) 
# iguais à "setup['start_date']".
start_date_filter = [df.index[0].date() == setup['start_date'] for df in filtered_data_array]

# Aplica o filtro criado acima, removendo assim todos os DataFrames presentes em "filtered_data_array" cujos índices iniciais são diferentes de 
# "setup['start_date']".
filtered_data_array = filtered_data_array[start_date_filter]

# Exibe uma mensagem informando quantos DataFrames não satisfaziam a condição do filtro criado acima, e informando também quantos DataFrames
# ainda existem em "data_array".
print(f"Foram removidos {filtered_data_array_initial_len - len(filtered_data_array)} tickers que não possuiam data inicial igual à {setup['start_date']}. Agora, todos os demais {len(filtered_data_array)} tickers \npossuem data de inicio em {setup['start_date']}.")

Foram removidos 13 tickers que não possuiam data inicial igual à 2019-01-02. Agora, todos os demais 484 tickers 
possuem data de inicio em 2019-01-02.


In [12]:
'''
    Essa célula será utilizada para mostrar que, nos dados baixados, não existem DataFrames em "data_array" cujos índices terminam em uma 
    data anterior à última data de negociação possível antes de setup['end_date'], isto é não existem DataFrames em "data_array" cujos índices
    terminam em uma data anterior à 2023-12-29.
'''

# Obtem qual é o menor dentre os últimos índices de cada um dos DataFrames presentes em "filtered_data_array"
lowest_last_date = np.min(np.array([df.index[len(df.index)-1] for df in filtered_data_array]))

# Exibe uma mensagem que informa qual é a menor data final entre todos os DataFrames presentes em "filtered_data_array" e gera uma
# conclusão a partir de tal data.
print(f"Dentre os tickers que passaram nos filtros acima, a menor data final é {lowest_last_date.date()}, que é a última data possível de negociação dentro do \nperiodo estabelecido na variável 'setup' ({setup['end_date']}). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.")

Dentre os tickers que passaram nos filtros acima, a menor data final é 2023-12-29, que é a última data possível de negociação dentro do 
periodo estabelecido na variável 'setup' (2023-12-30). Ou seja, nenhum filtro precisa ser aplicado em relação a data final dos tickers.


In [13]:
'''
    Essa célula verifica se algum dos DataFrames presentes em "filtered_data_array" possui valores nulos.
'''

# Cria uma lista auxiliar que conterá valores booleanos em todas as suas posições. Sendo que, para qualquer posição "i" dessa lista, 
# se tal posição contiver o valor True, indica que o DataFrame que ocupa essa mesma posição "i" na variável "filtered_data_array" 
# possui valores nulos. Caso contrário, indica que o DataFrame que ocupa tal posição em "filtered_data_array" não possui valores nulos.
aux_list = [df.isnull().values.any() for df in filtered_data_array]

# Exibe mensagens indicando se existem ou não DataFrames com valores nulos na variável "filtered_data_array".
if True in aux_list: 
    print("Existem DataFrames em 'filtered_data_array' que possuem valores nulos.")
else: 
    print("Não existem DataFrames em 'filtered_data_array' que possuam valores nulos.")

Não existem DataFrames em 'filtered_data_array' que possuam valores nulos.


In [14]:
'''
    Essa célula verifica se todos os DataFrames presentes em "filtered_data_array" possuem os mesmos índices.
'''

# Vamos tomar como referência os índices do DataFrame que ocupa a posição 0 da variável "filtered_data_array".
reference_indexes = filtered_data_array[0].index

# Verifica se todos os DataFrames presentes em "filtered_data_array" possuem os mesmos índices.
areAllIndexesEquals = all(df.index.equals(reference_indexes) for df in filtered_data_array)

if areAllIndexesEquals:
    print("Todos os DataFrames presentes em 'filtered_data_array' possuem os mesmos índices.")
else:
    print("Existem DataFrames presentes em 'filtered_data_array' que possuem índices distintos.")

Todos os DataFrames presentes em 'filtered_data_array' possuem os mesmos índices.


# **Feature Engineering**

#### *Gerando os retornos aritméticos diários dos tickers*

In [15]:
def get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos aritméticos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos aritméticos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Cria uma série temporal contendo os retornos diários do ticker em questão.
    ticker_daily_arithmetic_returns = daily_adjusted_closing_prices.pct_change()

    # Como não é possível calcular o retorno diário do "dia 0", removeremos a primeira linha da série temporal gerada acima.
    ticker_daily_arithmetic_returns = ticker_daily_arithmetic_returns[1:]
    
    # Retorna a série temporal resultante.
    return ticker_daily_arithmetic_returns

#### *Gerando os retornos logarítmicos diários dos tickers*

In [16]:
def get_ticker_daily_logarithmic_returns(daily_adjusted_closing_prices: pd.Series) -> pd.Series:
    '''
        Description:
            Essa função calcula e retorna a série temporal dos retornos logarítmicos diários de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
        Return:
            ticker_daily_arithmetic_returns (pd.Series): Série temporal dos retornos logarítmicos diários do ticker em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''

    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Obtem a série temporal dos retornos diários do ticker em questão
    ticker_daily_arithmetic_returns = get_ticker_daily_arithmetic_returns(daily_adjusted_closing_prices)
    
    # Transforma a série temporal dos retornos diários do ticker em questão em sua versão logarítmica.
    ticker_daily_logarithmic_returns = ticker_daily_arithmetic_returns.apply(lambda x: np.log(1 + x))
    
    # Retorna a série temporal resultante.
    return ticker_daily_logarithmic_returns

#### *Gerando uma métrica de volatilidade móvel*

In [17]:
def get_ticker_moving_volatility(daily_adjusted_closing_prices: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Essa função calcula a volatilidade móvel de "time_interval" dias de um certo ticker.
        Args:
            daily_adjusted_closing_prices (pd.Series): Série temporal dos preços de fechamento diários ajustados do ticker em questão.
            time_interval (int): Quantidade de dias da janela móvel.
        Return:
            moving_volatility (pd.Series): Série temporal que representa a evolução da volatilidade móvel de "time_interval" dias do ticker
                                           em questão.
        Errors:
            TypeError: É esperado que o parâmetro "daily_adjusted_closing_prices" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_adjusted_closing_prices" contenha apenas números.
    '''
    
    # Caso o parâmetro 'daily_adjusted_closing_prices' não seja um objeto do tipo pd.Series, um erro é retornado.
    if not isinstance(daily_adjusted_closing_prices, pd.Series):
        raise TypeError("É esperado que o parâmetro 'daily_adjusted_closing_prices' seja um objeto do tipo pd.Series.")

    # Caso o parâmetro 'time_interval' não seja do tipo int, um erro é retornado.
    if not isinstance(time_interval, int):
        raise TypeError("É esperado que o parâmetro 'time_interval' seja do tipo int.")
    
    # Caso a série temporal 'daily_adjusted_closing_prices' não seja numérica, um erro é retornado.
    if not(pd.api.types.is_numeric_dtype(daily_adjusted_closing_prices)):
        raise ValueError("É esperado que a série temporal 'daily_adjusted_closing_prices' contenha apenas números.")
    
    # Calcula a volatilidade móvel de "time_interval" dias da série temporal "daily_adjusted_closing_prices".
    moving_volatility = daily_adjusted_closing_prices.rolling(window=time_interval).std()
    
    # Remove os primeiros (time_interval - 1) dados da série temporal gerada acima, isto é, remove os dados NaN de tal série temporal.
    moving_volatility = moving_volatility[time_interval-1:]

    # Retorna o resultado obtido.
    return moving_volatility

#### *Gerando uma métrica de volume de negociação móvel*

In [18]:
def get_ticker_moving_volume(daily_volumes: pd.Series, time_interval: int) -> pd.Series:
    '''
        Description:
            Calcula o volume de negociação móvel de "time_interval" dias para uma série temporal de preços de um certo ticker.  
        Args:
            daily_volumes (pd.Series): Série temporal com os volumes de negociação diários do ticker em questão.
            time_interval (int): Número de dias para a janela móvel.
        Return:
            moving_volume (pd.Series): Série temporal do volume de negociação móvel calculado para o intervalo de dias especificado. 
        Errors:
            TypeError: É esperado que o parâmetro "daily_volumes" seja um objeto do tipo pd.Series.
            TypeError: É esperado que o parâmetro "time_interval" seja do tipo int.
            ValueError: É esperado que a série temporal "daily_volumes" contenha apenas números.
    '''
    
    # Verifica se o parâmetro 'daily_volumes' é do tipo pd.Series.
    if not isinstance(daily_volumes, pd.Series):
        raise TypeError("O parâmetro 'daily_volumes' deve ser do tipo pd.Series.")
    
    # Verifica se o parâmetro 'time_interval' é do tipo int.
    if not isinstance(time_interval, int):
        raise TypeError("O parâmetro 'time_interval' deve ser do tipo int.")
    
    # Verifica se a série temporal contém apenas dados numéricos.
    if not pd.api.types.is_numeric_dtype(daily_volumes):
        raise ValueError("A série temporal 'daily_volumes' deve conter apenas números.")
    
    # Calcula o volume de negociação móvel usando a janela de 'time_interval' dias.
    moving_volume = daily_volumes.rolling(window=time_interval).mean()
    
    # Remove os valores NaN resultantes do cálculo acima.
    moving_volume = moving_volume.dropna()

    return moving_volume


#### *Gerando algumas métricas de análise técnica (EMA , RSI , ATR , MOM) e adicionando novas features aos dados*

In [19]:
# Cria uma cópia do array "filtered_data_array" para inserir novas features sem eventual prejuizo as já existentes.
dfs = filtered_data_array

In [20]:
for df in dfs:
    # Calcula os retornos logarítmicos diários com base nos preços ajustados de fechamento e adiciona uma nova coluna 'Log Returns' ao DataFrame.
    df['Log Returns'] = get_ticker_daily_logarithmic_returns(df['Adj Close'])
    
    # Calcula a volatilidade móvel de 14 dias com base nos preços ajustados de fechamento e adiciona uma nova coluna 'MOV_VOL_14' ao DataFrame.
    df['MOV_VOLATILITY_14'] = get_ticker_moving_volatility(df['Adj Close'], 14)
    
    # Calcula o volume de negociação móvel de 14 dias com base nos preços ajustados de fechamento e adiciona uma nova coluna 'MOV_VOLUME_14' 
    # ao DataFrame.
    df['MOV_VOLUME_14'] = get_ticker_moving_volume(df['Volume'], 14)
    
    # Calcula a Média Móvel Exponencial (EMA) de 14 dias. A EMA é usada no lugar da Média Móvel Simples (SMA) para reduzir o atraso de sinal 
    # típico das médias móveis.
    # [Indicador de média móvel]
    df['EMA_14'] = talib.EMA(df['Adj Close'], timeperiod=14)
    
    # Calcula o Índice de Força Relativa (RSI) de 14 dias, que é um indicador de momentum utilizado para medir a velocidade e a mudança de
    # movimentos de preços.
    # [Indicador de momentum]
    df['RSI_14'] = talib.RSI(df['Adj Close'], timeperiod=14)
    
    # Calcula o Average True Range (ATR) de 14 dias, que é um indicador de volatilidade utilizado para medir a volatilidade do ativo com 
    # base na amplitude de preços (máxima, mínima e fechamento).
    # [Indicador de volatilidade]
    df['ATR_14'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)
    
    # Calcula o Momentum (MOM) de 14 dias, que é um indicador de momentum usado para identificar a força da tendência de um ativo.
    # [Indicador de momentum]
    df['MOM_14'] = talib.MOM(df['Adj Close'], timeperiod=14)
    
    # Adiciona uma coluna 'Ticker' ao DataFrame para identificar o ticker associado a cada conjunto de dados.
    df['Ticker'] = [df.name for _ in range(len(df))]
    
# Exibe o resultado obtido em um dos DataFrames presentes em "dfs"
dfs[0]

# Observação: valores NaN foram inseridos nos DataFrames juntamente com as novas features. Esses valores NaN são inerentes ao cálculo de alguns
# indicadores técnicos, uma vez que eles requerem um período de ajuste.

,Open,High,Low,Close,Adj Close,Volume,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,,,,,,
2019-01-02,157.040131,159.690628,156.103683,159.657196,126.860413,2960339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-03,157.424744,157.424744,152.918060,153.645493,122.083656,4016407,-0.038381,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-04,156.145493,160.518402,155.543472,159.966553,127.106224,3582140,0.040317,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-07,160.000000,160.785950,157.742477,159.598663,126.813904,2585991,-0.002302,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-08,161.371231,162.299332,158.511703,160.267563,127.345413,2965841,0.004182,NaN,NaN,NaN,NaN,NaN,NaN,MMM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,88.528427,89.807693,88.419731,88.904678,86.383263,2728913,0.007268,1.459183,4.854880e+06,84.524238,66.656526,1.665403,2.347855,MMM
2023-12-26,88.879601,90.794312,88.854515,90.392975,87.829346,3332176,0.016602,1.500868,4.830336e+06,84.964919,70.211804,1.685002,5.215652,MMM
2023-12-27,90.301003,91.220734,90.033447,90.919731,88.341164,2922785,0.005810,1.619461,4.733033e+06,85.415085,71.375161,1.649451,4.833817,MMM


#### *Eliminando redundâncias nas features e lidando com os valores nulos*

*Como o indicador ATR_14 é calculado a partir das variáveis "High" e "Low", essas duas variáveis se tornam redundantes para o treinamento do modelo. Além disso, utilizaremos a coluna "Adj Close" como variável alvo. Como "Adj Close" é uma versão ajustada da variável "Close", esta última também se torna redundante e pouco relevante como feature. Da mesma forma, sem a variável "Close", a variável "Open" perde parte de sua relevância. Ademais, temos duas features que medem aspectos semelhantes de maneiras diferentes: "Volume" e "MOV_VOLUME_14". Como a feature "MOV_VOLUME_14" parece ser mais relevante para o escopo deste projeto, a manutenção da feature "Volume" não é necessária. Portanto, remover essas features do conjunto de dados é uma boa abordagem.*

In [21]:
for df in dfs:
    # Remove as colunas 'Open', 'High', 'Low' e 'Close' de cada DataFrame presente em "dfs".
    df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)

# Exibe o resultado obtido em um dos DataFrames presentes em "dfs"
dfs[0]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-02,126.860413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-03,122.083656,-0.038381,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-04,127.106224,0.040317,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-07,126.813904,-0.002302,NaN,NaN,NaN,NaN,NaN,NaN,MMM
2019-01-08,127.345413,0.004182,NaN,NaN,NaN,NaN,NaN,NaN,MMM
...,...,...,...,...,...,...,...,...,...
2023-12-22,86.383263,0.007268,1.459183,4.854880e+06,84.524238,66.656526,1.665403,2.347855,MMM
2023-12-26,87.829346,0.016602,1.500868,4.830336e+06,84.964919,70.211804,1.685002,5.215652,MMM
2023-12-27,88.341164,0.005810,1.619461,4.733033e+06,85.415085,71.375161,1.649451,4.833817,MMM


*Observe que há dados faltantes nos DataFrames, principalmente devido ao período de ajuste dos indicadores técnicos. Como esses indicadores são calculados com uma janela de 14 dias, os primeiros 14 dias podem conter valores nulos. Para resolver isso, podemos remover as linhas com valores faltantes. Como estamos lidando com no máximo 14 linhas faltantes em um total de 1.258 linhas por DataFrame, essa remoção representa apenas uma pequena fração da amostra. Portanto, a quantidade de dados disponível para o treinamento do modelo não será significativamente afetada.*

In [22]:
for df in dfs:
    # Remove todas as linhas com valores NaN de cada DataFrame na lista "dfs".
    df.dropna(inplace=True)

# Exibe o resultado obtido em um dos DataFrames presentes em "dfs"
dfs[0]

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-23,127.750641,0.000156,1.812780,2.878080e+06,127.136215,52.022901,3.724319,0.890228,MMM
2019-01-24,127.903458,0.001195,1.099697,2.726410e+06,127.238514,52.379070,3.557437,5.819801,MMM
2019-01-25,130.149033,0.017404,1.294803,2.668746e+06,127.626583,57.385431,3.580450,3.042809,MMM
2019-01-28,128.355225,-0.013879,1.275540,2.753696e+06,127.723735,52.625973,3.613165,1.541321,MMM
2019-01-29,130.846603,0.019224,1.493873,2.910142e+06,128.140118,57.854245,3.754628,3.501190,MMM
...,...,...,...,...,...,...,...,...,...
2023-12-22,86.383263,0.007268,1.459183,4.854880e+06,84.524238,66.656526,1.665403,2.347855,MMM
2023-12-26,87.829346,0.016602,1.500868,4.830336e+06,84.964919,70.211804,1.685002,5.215652,MMM
2023-12-27,88.341164,0.005810,1.619461,4.733033e+06,85.415085,71.375161,1.649451,4.833817,MMM


####

# **Filtrando as ações**

Para esse projeto, buscaremos trabalhar com ações que possuam, para um período de 14 dias, alto volume de negociação e baixa volatilidade. Para tal, filtraremos todas as ações obtidas através da métrica $V$ abaixo:

$$V = \dfrac{V_{l}}{V_{t}}$$

Onde,

$V_{l} =$ Volume móvel de 14 dias.

$V_{t} =$ Volatilidade móvel de 14 dias.

In [23]:
# Cria um DataFrame vazio para armazenar os resultados.
results = pd.DataFrame()  

for df in dfs:  
    # Obtém o último valor da feature 'MOV_VOLUME_14' do DataFrame em questão.
    df_mov_volume = df.tail(1)['MOV_VOLUME_14']  
    
    # Obtém o último valor da feature 'MOV_VOLATILITY_14' do DataFrame em questão.
    df_mov_volatility = df.tail(1)['MOV_VOLATILITY_14']  
    
    # Obtem o nome do ticker que representa os dados do DataFrame em questão.
    df_ticker = df.tail(1)['Ticker'] 
    
    # Calcula o 'V_Value' do DataFrame em questão.
    df_v_value = df_mov_volume / df_mov_volatility  
    
    # Cria um DataFrame auxiliar com os valores calculados.
    aux_df = pd.DataFrame({  
        "V_Value": df_v_value,
        "Ticker": df_ticker
    })
    
    # Concatena o DataFrame auxiliar com o DataFrame de resultados, acumulando os dados e gerando o resultado final ao final do loop.
    results = pd.concat([results, aux_df])  

In [24]:
# Ordena o DataFrame 'results' com base na coluna 'V_Value' em ordem crescente.
results = results.sort_values(by="V_Value")  

# Exibe o DataFrame ordenado.
results

,V_Value,Ticker
Date,,
2023-12-29,1.393141e+02,NVR
2023-12-29,3.201976e+03,AZO
2023-12-29,3.805557e+03,MTD
2023-12-29,4.296223e+03,BKNG
2023-12-29,1.464196e+04,FICO
...,...,...
2023-12-29,1.037057e+08,AMCR
2023-12-29,1.110892e+08,AAL
2023-12-29,1.149379e+08,F


In [25]:
# Seleciona os últimos 'tickers_to_select' valores da coluna 'Ticker' do DataFrame 'results' e converte esses valores em um array NumPy. 
# Lembre que 'setup['tickers_to_select']' define o número de tickers a serem selecionados.
selected_tickers = results.tail(setup['tickers_to_select'])['Ticker'].to_numpy()

# Exibe o array de tickers selecionados.
selected_tickers

array(['NVDA'], dtype=object)

In [26]:
# Cria uma lista vazia para armazenar o resultado final (DataFrames que passaram pelo filtro).
data_list = []

# Itera sobre cada um dos DataFrames da variável "dfs".
for df in dfs: 
    # Verifica se o nome do DataFrame está na lista de tickers selecionados.
    if df.name in selected_tickers:
        # Guarda o DataFrame que contém os dados do ticker em questão em "data_list".
        data_list.append(df)
     
# Exibe o resultado obtido (isto é, os DataFrames com dados de todos os tickers que passaram pelo filtro).
data_list

[            Adj Close  Log Returns  MOV_VOLATILITY_14  MOV_VOLUME_14  \
 Date                                                                   
 2019-01-23   3.702781     0.003489           0.181737   6.440111e+08   
 2019-01-24   3.914844     0.055691           0.148750   6.442554e+08   
 2019-01-25   3.972138     0.014529           0.144687   6.848949e+08   
 2019-01-28   3.423009    -0.148785           0.159213   8.136354e+08   
 2019-01-29   3.264023    -0.047559           0.188673   8.399151e+08   
 ...               ...          ...                ...            ...   
 2023-12-22  48.823704    -0.003271           1.299941   3.922356e+08   
 2023-12-26  49.272640     0.009153           1.278714   3.831272e+08   
 2023-12-27  49.410622     0.002796           1.065473   3.726314e+08   
 2023-12-28  49.515610     0.002123           0.959779   3.651859e+08   
 2023-12-29  49.515610     0.000000           0.927387   3.673337e+08   
 
                EMA_14     RSI_14    ATR_14    MO

# **Data Preparation**

#### *Preparando o DataFrame que será usado como exemplo*

In [27]:
# Para esse projeto, vamos tentar prever o valor de apenas um ticker. Para tal, pegaremos o ticker da primeira posição de "data_list"

# Obtem o DataFrame do ticker que tentaremos prever o valor com o modelo LSTM.
ticker_data = data_list[0]

# Exibe tal DataFrame
ticker_data

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14,Ticker
Date,,,,,,,,,
2019-01-23,3.702781,0.003489,0.181737,6.440111e+08,3.605714,61.630178,0.173286,0.324170,NVDA
2019-01-24,3.914844,0.055691,0.148750,6.442554e+08,3.646931,67.032488,0.177444,0.740358,NVDA
2019-01-25,3.972138,0.014529,0.144687,6.848949e+08,3.690292,68.329865,0.181876,0.594270,NVDA
2019-01-28,3.423009,-0.148785,0.159213,8.136354e+08,3.654654,48.592084,0.220939,-0.133686,NVDA
2019-01-29,3.264023,-0.047559,0.188673,8.399151e+08,3.602570,44.577244,0.217818,-0.204126,NVDA
...,...,...,...,...,...,...,...,...,...
2023-12-22,48.823704,-0.003271,1.299941,3.922356e+08,48.316349,55.352140,1.346709,3.323582,NVDA
2023-12-26,49.272640,0.009153,1.278714,3.831272e+08,48.443854,57.245976,1.305515,2.712650,NVDA
2023-12-27,49.410622,0.002796,1.065473,3.726314e+08,48.572757,57.837925,1.254764,3.913498,NVDA


In [28]:
# Guarda o nome do ticker selecionado.
ticker_name = ticker_data.name

# Remove a coluna "Ticker" pois ela não será necessária no treinamento do modelo.
ticker_data.drop(['Ticker'], axis=1, inplace=True)

# Exibe o DataFrame resultante
ticker_data

,Adj Close,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14
Date,,,,,,,,
2019-01-23,3.702781,0.003489,0.181737,6.440111e+08,3.605714,61.630178,0.173286,0.324170
2019-01-24,3.914844,0.055691,0.148750,6.442554e+08,3.646931,67.032488,0.177444,0.740358
2019-01-25,3.972138,0.014529,0.144687,6.848949e+08,3.690292,68.329865,0.181876,0.594270
2019-01-28,3.423009,-0.148785,0.159213,8.136354e+08,3.654654,48.592084,0.220939,-0.133686
2019-01-29,3.264023,-0.047559,0.188673,8.399151e+08,3.602570,44.577244,0.217818,-0.204126
...,...,...,...,...,...,...,...,...
2023-12-22,48.823704,-0.003271,1.299941,3.922356e+08,48.316349,55.352140,1.346709,3.323582
2023-12-26,49.272640,0.009153,1.278714,3.831272e+08,48.443854,57.245976,1.305515,2.712650
2023-12-27,49.410622,0.002796,1.065473,3.726314e+08,48.572757,57.837925,1.254764,3.913498


#### *Dividindo os dados em treino e teste*

In [29]:
# Separa as features e o target
X = ticker_data.drop(columns=['Adj Close'])
y = ticker_data['Adj Close']

In [30]:
# Exibe as features
X

,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14
Date,,,,,,,
2019-01-23,0.003489,0.181737,6.440111e+08,3.605714,61.630178,0.173286,0.324170
2019-01-24,0.055691,0.148750,6.442554e+08,3.646931,67.032488,0.177444,0.740358
2019-01-25,0.014529,0.144687,6.848949e+08,3.690292,68.329865,0.181876,0.594270
2019-01-28,-0.148785,0.159213,8.136354e+08,3.654654,48.592084,0.220939,-0.133686
2019-01-29,-0.047559,0.188673,8.399151e+08,3.602570,44.577244,0.217818,-0.204126
...,...,...,...,...,...,...,...
2023-12-22,-0.003271,1.299941,3.922356e+08,48.316349,55.352140,1.346709,3.323582
2023-12-26,0.009153,1.278714,3.831272e+08,48.443854,57.245976,1.305515,2.712650
2023-12-27,0.002796,1.065473,3.726314e+08,48.572757,57.837925,1.254764,3.913498


In [31]:
# Exibe o target
y

Date
2019-01-23     3.702781
2019-01-24     3.914844
2019-01-25     3.972138
2019-01-28     3.423009
2019-01-29     3.264023
                ...    
2023-12-22    48.823704
2023-12-26    49.272640
2023-12-27    49.410622
2023-12-28    49.515610
2023-12-29    49.515610
Name: Adj Close, Length: 1244, dtype: float64

In [32]:
# Separa as o conjunto de séries temporais das features em dados de treino e dados de teste.
X_train, X_test = X[X.index < setup['test_initial_day']], X[X.index >= setup['test_initial_day']]
# Separa a série temporal do target em dados de treino e dados de teste.
y_train, y_test = y[y.index < setup['test_initial_day']], y[y.index >= setup['test_initial_day']]

In [33]:
# Exibe o conjunto de séries temporais das features que será usado para o treinamento do modelo.
X_train

,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14
Date,,,,,,,
2019-01-23,0.003489,0.181737,6.440111e+08,3.605714,61.630178,0.173286,0.324170
2019-01-24,0.055691,0.148750,6.442554e+08,3.646931,67.032488,0.177444,0.740358
2019-01-25,0.014529,0.144687,6.848949e+08,3.690292,68.329865,0.181876,0.594270
2019-01-28,-0.148785,0.159213,8.136354e+08,3.654654,48.592084,0.220939,-0.133686
2019-01-29,-0.047559,0.188673,8.399151e+08,3.602570,44.577244,0.217818,-0.204126
...,...,...,...,...,...,...,...
2023-11-24,-0.019484,1.495081,4.405599e+08,47.940908,55.058021,1.471438,2.770397
2023-11-27,0.009707,1.302960,4.401976e+08,47.979658,56.921857,1.429050,2.490459
2023-11-28,-0.008765,1.104189,4.443638e+08,47.957120,54.714159,1.387689,1.865589


In [34]:
# Exibe o conjunto de séries temporais das features que será usado para o teste do modelo.
X_test

,Log Returns,MOV_VOLATILITY_14,MOV_VOLUME_14,EMA_14,RSI_14,ATR_14,MOM_14
Date,,,,,,,
2023-12-01,-0.000107,1.105444,4.421573e+08,47.675736,49.108316,1.355323,-1.569660
2023-12-04,-0.027203,1.381042,4.459721e+08,47.385655,43.383999,1.383871,-3.109329
2023-12-05,0.023027,1.408858,4.427409e+08,47.275566,48.811807,1.379952,-3.085236
2023-12-06,-0.023092,1.557511,4.359619e+08,47.038440,44.232132,1.422455,-3.380272
2023-12-07,0.023737,1.526196,4.367524e+08,46.978647,49.480688,1.401280,-2.879280
2023-12-08,0.019341,1.457193,4.391823e+08,47.048143,53.411962,1.386260,-1.787437
2023-12-11,-0.018676,1.227006,4.460114e+08,46.991189,49.412124,1.408742,-3.777081
2023-12-12,0.021850,0.978276,4.322000e+08,47.079144,53.779973,1.423831,-2.282314
2023-12-13,0.009003,0.914007,3.999409e+08,47.212831,55.510984,1.392558,-0.623631


In [35]:
# Exibe a parte da série temporal do target que será utilizada para o treino do modelo.
y_train

Date
2019-01-23     3.702781
2019-01-24     3.914844
2019-01-25     3.972138
2019-01-28     3.423009
2019-01-29     3.264023
                ...    
2023-11-24    47.765636
2023-11-27    48.231533
2023-11-28    47.810623
2023-11-29    48.129559
2023-11-30    46.759853
Name: Adj Close, Length: 1224, dtype: float64

In [36]:
# Exibe a parte da série temporal do target que será utilizada para o teste do modelo.
y_test

Date
2023-12-01    46.754852
2023-12-04    45.500122
2023-12-05    46.559990
2023-12-06    45.497124
2023-12-07    46.589989
2023-12-08    47.499870
2023-12-11    46.620983
2023-12-12    47.650852
2023-12-13    48.081799
2023-12-14    48.343758
2023-12-15    48.883690
2023-12-18    50.070538
2023-12-19    49.597599
2023-12-20    48.104790
2023-12-21    48.983681
2023-12-22    48.823704
2023-12-26    49.272640
2023-12-27    49.410622
2023-12-28    49.515610
2023-12-29    49.515610
Name: Adj Close, dtype: float64

#### *Normalizando as features*

In [37]:
# Cria uma instancia do MinMaxScaler para as features.
scaler_features = MinMaxScaler()

# Ajusta a instância criado ao conjunto de treino das features.
X_train_scaled = scaler_features.fit_transform(X_train)  

# Redimensiona y_train para que ele seja bidimensional (Necessário para o MinMaxScaler).
resized_y_train = y_train.values.reshape(-1, 1)  

# Cria uma instancia do MinMaxScaler para o target.
scaler_target = MinMaxScaler()

# Ajusta a instância criada ao conjunto de treino do target.
y_train_scaled = scaler_target.fit_transform(resized_y_train)  

# Normaliza o conjunto de teste das features usando a instância que foi criada e ajustada aos dados de treino.
X_test_scaled = scaler_features.transform(X_test)  

# Redimensiona y_test para que seja bidimensional (Necessário para o MinMaxScaler).
resized_y_test = y_test.values.reshape(-1, 1) 

# Normaliza o conjunto de teste do target usando a instância que foi criada e ajustada aos dados de treino.
y_test_scaled = scaler_target.transform(resized_y_test)

In [38]:
# Exibe o resultado da normalização das features do conjunto de treino.
X_train_scaled

array([[0.49155292, 0.03111757, 0.64608127, ..., 0.60802108, 0.0304872 ,
        0.41635709],
       [0.61523506, 0.0236745 , 0.64643526, ..., 0.68491016, 0.03260545,
        0.43637524],
       [0.51770951, 0.02275781, 0.70532497, ..., 0.70337525, 0.03486348,
        0.42934861],
       ...,
       [0.46251908, 0.23925856, 0.35677662, ..., 0.50958792, 0.64912826,
        0.49049746],
       [0.49903888, 0.20430267, 0.36042892, ..., 0.52936101, 0.6328746 ,
        0.47607135],
       [0.41488131, 0.21036067, 0.35895407, ..., 0.4301435 , 0.64605458,
        0.39210898]])

In [39]:
# Exibe o resultado da normalização do target do conjunto de treino.
y_train_scaled

array([[0.00930873],
       [0.01380788],
       [0.01502342],
       ...,
       [0.94510463],
       [0.95187119],
       [0.92281141]])

#### Ajustando os dados para o formato aceito por modelos LSTM

In [40]:
def create_sequences(features_array: np.array, target_array: np.array, sequence_length: int) -> tuple[np.ndarray, np.ndarray]:
    '''
        Description:
            Essa função cria e retorna sequências temporais de tamanho fixo para treinamento de modelos de aprendizado de máquina,
            como redes neurais LSTM. Cada sequência de entrada é composta por 'sequence_length' dias consecutivos de dados,
            e o valor alvo é o preço de fechamento ajustado ('Adj Close') do dia seguinte.
        Args:

        Return:
            sequences (np.ndarray): Array NumPy contendo as sequências de entrada (X) para o modelo.
            targets (np.ndarray): Array NumPy contendo os valores de saída (y) correspondentes a serem previstos.
        Errors:

    '''

    sequences = []  # Lista para armazenar as sequências de entrada (X)
    targets = []    # Lista para armazenar os valores de saída (y)

    # Itera sobre o DataFrame/data para criar as sequências de tamanho 'sequence_length'
    for i in range(len(features_array) - sequence_length):
        # Cria uma sequência de 'sequence_length' dias
        seq = features_array[i:i + sequence_length]  # Obtém uma sequência de 'sequence_length' dias

        # O alvo será o 'Adj Close' do próximo dia após a sequência
        target = target_array[i + sequence_length]  # Obtém o valor de 'Adj Close' no próximo dia após a sequência

        sequences.append(seq)  # Adiciona a sequência à lista de sequências
        targets.append(target)  # Adiciona o valor alvo à lista de alvos

    # Retorna as listas convertidas para arrays NumPy
    return np.array(sequences), np.array(targets)

In [41]:
#
sequence_length = 7

X_train_scaled_sequence,y_train_scaled_sequence = create_sequences(X_train_scaled, y_train_scaled, sequence_length)

In [42]:
#
X_train_scaled_sequence

array([[[0.49155292, 0.03111757, 0.64608127, ..., 0.60802108,
         0.0304872 , 0.41635709],
        [0.61523506, 0.0236745 , 0.64643526, ..., 0.68491016,
         0.03260545, 0.43637524],
        [0.51770951, 0.02275781, 0.70532497, ..., 0.70337525,
         0.03486348, 0.42934861],
        ...,
        [0.37060448, 0.03268258, 0.92996149, ..., 0.36531292,
         0.05317293, 0.39094671],
        [0.58529983, 0.03508443, 0.96690968, ..., 0.4239951 ,
         0.05116915, 0.39457339],
        [0.59050147, 0.03537408, 1.        , ..., 0.48305129,
         0.05140077, 0.39899931]],

       [[0.61523506, 0.0236745 , 0.64643526, ..., 0.68491016,
         0.03260545, 0.43637524],
        [0.51770951, 0.02275781, 0.70532497, ..., 0.70337525,
         0.03486348, 0.42934861],
        [0.13077251, 0.02603547, 0.89188013, ..., 0.42245467,
         0.05476263, 0.39433483],
        ...,
        [0.58529983, 0.03508443, 0.96690968, ..., 0.4239951 ,
         0.05116915, 0.39457339],
        [0.5

In [43]:
#
X_train_scaled_sequence.shape #1210 amostras, com 7 dias cada uma, e com 7 features representadas por dia

(1217, 7, 7)

In [44]:
#
y_train_scaled_sequence

array([[0.0069092 ],
       [0.00925085],
       [0.00965604],
       ...,
       [0.94510463],
       [0.95187119],
       [0.92281141]])

In [45]:
X_test_scaled_sequence, y_test_scaled_sequence = create_sequences(X_test_scaled, y_test_scaled,sequence_length)

In [46]:
X_test_scaled_sequence

array([[[0.48303288, 0.23954152, 0.35357923, 0.99314547, 0.42980201,
         0.63264029, 0.32526623],
        [0.41883442, 0.30172719, 0.35910716, 0.98661283, 0.34832991,
         0.64718343, 0.25121007],
        [0.53784293, 0.30800345, 0.35442498, 0.98413364, 0.42558191,
         0.64518677, 0.25236894],
        [0.42857352, 0.3415453 , 0.34460157, 0.97879358, 0.36040107,
         0.66683895, 0.23817806],
        [0.53952512, 0.3344796 , 0.34574707, 0.97744703, 0.43510184,
         0.65605191, 0.26227514],
        [0.52911143, 0.31890976, 0.34926823, 0.97901209, 0.49105421,
         0.64840048, 0.31479142],
        [0.43903682, 0.26697081, 0.35916419, 0.97772947, 0.434126  ,
         0.65985282, 0.21909206]],

       [[0.41883442, 0.30172719, 0.35910716, 0.98661283, 0.34832991,
         0.64718343, 0.25121007],
        [0.53784293, 0.30800345, 0.35442498, 0.98413364, 0.42558191,
         0.64518677, 0.25236894],
        [0.42857352, 0.3415453 , 0.34460157, 0.97879358, 0.36040107,
  

In [47]:
X_test_scaled_sequence.shape

(13, 7, 7)

In [48]:
y_test_scaled_sequence

array([[0.94171492],
       [0.95085791],
       [0.95641566],
       [0.96787091],
       [0.99305118],
       [0.98301727],
       [0.9513457 ],
       [0.96999233],
       [0.96659824],
       [0.97612292],
       [0.97905034],
       [0.98127778],
       [0.98127778]])

In [49]:
# Defina o número de features
n_features = X_test.shape[1]

# Definindo o modelo LSTM
model = Sequential()

# Camada LSTM
model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, n_features)))
model.add(Dropout(0.2))  # Regularização para evitar overfitting

model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Camada de saída
model.add(Dense(units=1))

# Compilando o modelo
model.compile(optimizer='adam', loss='mean_squared_error')


/home/user/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [58]:
# Treinando o modelo
history = model.fit(X_train_scaled_sequence, y_train_scaled_sequence, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 3.7890e-04 - val_loss: 0.0056
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 3.6816e-04 - val_loss: 0.0061
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.0307e-04 - val_loss: 0.0056
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 3.6241e-04 - val_loss: 0.0059
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 3.1683e-04 - val_loss: 0.0046
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.9445e-04 - val_loss: 0.0063
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 3.3195e-04 - val_loss: 0.0036
Epoch 8/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 3.2347e-04 - val_loss: 0.0046
Epoch 9/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 2.7841e-04 - val_loss: 0.0058
Epoch 10/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 3.4620e-04 - val_loss: 0.0046
Epoch 11/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.0040e-04 - val_loss: 0.0054
Epoch 12/50
31/31 ━

In [51]:
# Previsão
predicted_prices = model.predict(X_test_scaled_sequence)

# Avaliação (por exemplo, usando RMSE)
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_test_scaled_sequence, predicted_prices))
print(f"RMSE: {rmse}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 647ms/step
RMSE: 0.13639270829614186


In [52]:
def plot_multiples_time_series_line_graphs(data_list: list, serie_title: Optional[str] = "", serie_xaxis_title: Optional[str] = "", 
                               serie_yaxis_title: Optional[str] = "") -> None:
    '''
        Description:
            Esta função exibe em um mesmo plot múltiplos gráficos de séries temporais. 
        Args:
            data_list (list): Lista contendo as séries temporais cujos gráficos serão exibidos no plot. Os valores de tais séries temporais
                              estarão no eixo y, enquanto os índices dessas séries estarão no eixo x.
            title (string) [Optional]: Título do plot.
            xaxis_title (string) [Optional]: Título do eixo x do plot.
            yaxis_title (string) [Optional]: Título do eixo y do plot.
        Return:
            None: A função exibe os gráficos, mas não retorna nenhum valor.  
        Errors:
            TypeError: É esperado que todas os elementos da lista "data_list" sejam objetos do tipo pd.Series, isto é, que sejam séries temporais.
            ValueError: É esperado que todas as séries temporais presentes na variável "data_list" possuam os mesmos índices. 
            TypeError: É esperado que todas as séries temporais presentes na variável data_list possuam um atributo "name".
    '''    

    # Verifica se todos os elementos presentes na lista "data_list" são séries temporais.
    are_all_data_time_series = all(isinstance(df, pd.Series) for df in data_list)
    
    # Retorna um erro caso algum dos dados presentes na variável "data_list" não seja uma série temporal
    if not are_all_data_time_series:
        raise TypeError("Todos os dados presentes no parâmetro 'data_list' devem ser séries temporais.")
    
    # Verifica se as séries temporais presentes na variável "data_list" possuem os mesmos índices.
    are_all_index_equal = all(df.index.equals(data_list[0].index) for df in data_list)
    
    # Retorna um erro caso as séries temporais possuam índices diferentes.
    if not are_all_index_equal:
        raise ValueError("Todos as séries temporais devem possuir os mesmos índices.")
    
    # Verifica se todas as séries temporais presentes na variável "data_list" possuem o atributo "name".
    all_time_series_have_names = all(hasattr(df,"name") for df in data_list)
    
    # Retorna um erro caso uma das séries temporais presentes na variável "data_list" não possua o atributo "name".
    if not all_time_series_have_names:
        raise TypeError("Todas as séries temporais devem possuir o atributo 'name'")
    
    # Cria uma lista de timestamps que representará o eixo x do gráfico que será plotado.
    x_axis = data_list[0].index.tolist() # Observe que só podemos fazer isso pois temos certeza que todas as séries temporais possuem os mesmos índices.
    # Cria a figura onde será plotado o gráfico.
    fig = go.Figure()

    # Adiciona cada série temporal ao gráfico.
    for time_serie in data_list:
        # Plota o gráfico (Data x Valor da Ação) do ticker em questão.
        fig.add_trace(go.Scatter(x=x_axis, y=time_serie.values, mode="lines", name=time_serie.name))
    
    # Atualiza o layout para permitir destaque ao clicar na legenda.
    fig.update_layout(
        # Seta um título para o plot.
        title=serie_title,
        # Seta um título para o eixo x do plot.
        xaxis_title=serie_xaxis_title,
        # Seta um título para o eixo y do plot.
        yaxis_title=serie_yaxis_title,   
    )
    
    # Exibe o gráfico criado.
    fig.show()

In [53]:
real_prices_time_series = pd.Series(
    scaler_target.inverse_transform(y_test_scaled_sequence).flatten(), 
    index=y_test.index[len(y_test) - len(y_test_scaled_sequence):]
)

real_prices_time_series.name = 'Preço real'

In [54]:
real_prices_time_series

Date
2023-12-12    47.650852
2023-12-13    48.081799
2023-12-14    48.343758
2023-12-15    48.883690
2023-12-18    50.070538
2023-12-19    49.597599
2023-12-20    48.104790
2023-12-21    48.983681
2023-12-22    48.823704
2023-12-26    49.272640
2023-12-27    49.410622
2023-12-28    49.515610
2023-12-29    49.515610
Name: Preço real, dtype: float64

In [55]:
predicted_prices_time_series = pd.Series(
    scaler_target.inverse_transform(predicted_prices).flatten(),
    index=y_test.index[len(y_test) - len(y_test_scaled_sequence):]
)

predicted_prices_time_series.name = 'Preço estimado'

In [56]:
predicted_prices_time_series

Date
2023-12-12    40.472332
2023-12-13    41.436905
2023-12-14    41.891792
2023-12-15    42.318970
2023-12-18    42.625118
2023-12-19    43.445572
2023-12-20    42.978909
2023-12-21    42.461136
2023-12-22    42.976109
2023-12-26    42.877769
2023-12-27    43.074127
2023-12-28    43.275059
2023-12-29    43.145405
Name: Preço estimado, dtype: float32

In [57]:
plot_multiples_time_series_line_graphs([real_prices_time_series, predicted_prices_time_series], "Preço real x Preço estimado", "Data", "Preço")

###